# MSD Functions

MSD can be de composed into the two groupings described in the link above. The second grouping can be shown to be the autocorrelation function of the time series (a wide-sense-stationary process), which according to the Weiner-Khinchin theorem, can be found quickly from the spectral decomposition of the underlying stochastic process. (see Wikipedia)

#https://stackoverflow.com/questions/31264591/mean-square-displacement-python
#https://stackoverflow.com/questions/34222272/computing-mean-square-displacement-using-python-and-fft

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [5]:
#%run ./Combined-fxns-single-2D.ipynb

# Overlapping Increments (Weiner-Kinchin Thm)

In [6]:
def autocorrFFT(x):
    
    #length of time series
    N=len(x)
    
    #compute 1-D fast Fourier transform of a time series
    F = np.fft.fft(x, n=2*N)  #2*N because of zero-padding
    
    #define power spectral density, which is magnitude of fft
    PSD = F * F.conjugate()
    
    #take the inverse Fourier transform of the PSD
    res = np.fft.ifft(PSD)
    
    #take the real portion only
    res= (res[:N]).real   #now we have the autocorrelation in convention B
    
    #divide res(m) by (N-m)
    n=N*np.ones(N)-np.arange(0,N) 
    return res/n

In [7]:
def msd_fft(r):
    
    #length of time series
    N=len(r)
    
    #squares and sums x positions and y positions (columns)
    D=np.square(r).sum(axis=1) 
    
    #adds on zero
    D=np.append(D,0) 
    
    #computes autocorr fxn of each dimension of time series with PSD FFT method
    S2=sum([autocorrFFT(r[:, i]) for i in range(r.shape[1])])
    
    #set up for computing S1 from D
    Q=2*D.sum()
    
    #initialize S1
    S1=np.zeros(N)
    
    #compute S1 recursively
    for m in range(N):
        Q=Q-D[m-1]-D[N-m]
        S1[m]=Q/(N-m)
        
    #compute the mean of the squared displacement (S1-2*S2) by dividing by nlags    
    return S1-2*S2

# Independent Increments
Note that strange looking jumps in MSD occur as a side effect of the sample size reducing

In [1]:
#non overlapping intervals, takes in two dimensional path
#starts increments from bottom of time series
def difference_indep(dataset, interval):
    
    diff = list()
    data = np.asarray(pd.concat([dataset['X'],dataset['Y']],axis=1))
    
    # see if difference when to lag starting at latest t
    if (len(dataset) % interval) == 0:
        X=0
    else:
        X=1
        
    for i in range(1,int(np.floor(len(dataset)/interval))+X):
        value = data[i*interval,:] - data[(i-1)*interval,:]
        #print(i*interval,(i-1)*interval)
        diff.append(value)
        
    return (diff)

In [1]:
#non overlapping intervals, takes in two dimensional path, 
#starts increments from top of time serires 
def difference_indep2(dataset, interval):
    
    diff = list()
    data = np.asarray(pd.concat([dataset['X'],dataset['Y']],axis=1))
    #print(data)
    #print(dataset)
    
    if (len(dataset) % interval) == 0:
        X=1
    else:
        X=0
    k=0    

    idx_top = int(len(dataset))-1
    idx_bottom = 0
    #print('top=',idx_top)


    for i in range( int(len(dataset)/interval)-X,0,-1):
        #print(i)
        value = data[idx_top-(i-1)*(interval),:] - data[idx_top-i*(interval),:]
        #print(idx_top-(i-1)*(interval),idx_top-i*(interval))
        diff.append(value)
        
    return (diff)

In [3]:
def msd_indep(dataset):
    #msd = np.zeros(int(len(dataset)/2))
    msd = np.zeros(int(len(dataset)-1))

    #for i in range(1,int(len(dataset)/2)):
    for i in range(1,int(len(dataset)-1)):

        sq_inc = np.square(difference_indep(dataset,interval=i)).sum(axis=1)
        final = sq_inc.mean()
        msd[i] = final

    return(msd)

In [4]:
def msd_indep2(dataset):
    #msd = np.zeros(int(len(dataset)/2))
    msd = np.zeros(int(len(dataset)-1))

    #for i in range(1,int(len(dataset)/2)):
    for i in range(1,int(len(dataset)-1)):

        sq_inc = np.square(difference_indep2(dataset,interval=i)).sum(axis=1)
        final = sq_inc.mean()
        msd[i] = final

    return(msd)

In [1]:
# def maybe_msd(tpts, V, tau):
#     msd = []
#     for i in range(1,len(tpts)):
#         value = 2*((tau*V)**2)*(tpts[i]/tau + np.exp(-tpts[i]/tau)-1)
#         msd.append(value)
#     return(msd)

In [1]:
def abp_msd(tpts, V, tau, eps):
    msd = []
    for i in range(1,len(tpts)):
        if(tau>100000000):
            value =  4*(V**2)*tpts[i] +2*(eps**2)*tpts[i] #if tau too big, set exponential =1
        else:
            # value = 2*((tau*V)**2)*(tpts[i]/tau + np.exp(-tpts[i]/(tau))-1) + 2*(eps**2)*tpts[i]
            value = 4*((tau*V)**2)*(tpts[i]/tau + 2*(np.exp(-tpts[i]/(2*tau))-1)) + 2*(eps)*tpts[i]
        msd.append(value)
    if(tau>100000000):
        print('limiting MSD used')
    return(msd)

In [168]:
# interval=10
# dataset=path

# if (len(dataset) % interval) == 0:
#     X=1
# else:
#     X=0
# k=0    

# idx_top = int(len(dataset))-1
# idx_bottom = 0
# print(idx_top,idx_bottom)

# for i in range( int(len(dataset)/interval)-X,0,-1):
#     k=k+1
#     print(idx_top-(i-1)*(interval),idx_top-i*(interval))


# print(k)
